In [1]:
from GSNN import utils
import numpy as np
import scipy.sparse as sp
from tqdm import tqdm
import torch

from GSNN.GSNNmodel import GSNN, GCN
from GSNN.GSNNTrain import GSNNTrainer, GCNTrainer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:
dataset = 'cora'
seed = 15
ratio = 0.05 # ratio of training set
unlabeled_share = 0.8
train_share = ratio 
val_share = 1 - unlabeled_share - train_share
weight_decay = 5e-4
lr = 1e-2
hidden_dim = 16
attack = None
epochs = 200
earlystopping = False

In [3]:
_A_obs, _X_obs, _z_obs = utils.load_npz('data/' + dataset + '.npz')

if _X_obs is None:
    _X_obs = sp.eye(_A_obs.shape[0]).tocsr()
lcc = utils.largest_connected_components(_A_obs)


if attack=='meta_self':
    _A_obs = sp.csr_matrix(np.load('data/' + dataset + '_0.05edges_Meta-Self.npy'))
elif attack=='meta_train':
    _A_obs = sp.csr_matrix(np.load('data/' + dataset + '_0.05edges_Meta-Train.npy'))
elif attack=='minmax':
    _A_obs = sp.csr_matrix(np.load('data/' + dataset + '_0.05_minmax.npy'))
else:
    _A_obs = _A_obs[lcc][:,lcc]

_A_obs = _A_obs + _A_obs.T
_A_obs[_A_obs > 1] = 1


_A_obs.setdiag(0)
_A_obs = _A_obs.astype("float32")
_A_obs.eliminate_zeros()
_X_obs = _X_obs.astype("float32")

_X_obs = _X_obs[lcc]
_z_obs = _z_obs[lcc]
_N = _A_obs.shape[0]
_K = _z_obs.max()+1
_Z_obs = np.eye(_K)[_z_obs]

#normalized adj
_An = utils.preprocess_graph(_A_obs)
degrees = _A_obs.sum(0).A1

np.random.seed(seed)

split_train, split_val, split_test = utils.train_val_test_split_tabular(np.arange(_N),
                                                                    train_size=train_share,
                                                                    val_size=val_share,
                                                                    test_size=unlabeled_share,
                                                                    stratify=_z_obs)
print('Number of trainging set: {}'.format(split_train.shape[0]))
split_unlabeled = np.union1d(split_val, split_test)

utils.set_seed(42, torch.cuda.is_available())
adj = utils.sparse_mx_to_torch_sparse_tensor(_An).float()
adj_cuda = adj.cuda()

feature_cuda, label_cuda, idx_train_cuda, idx_val_cuda, idx_test_cuda = utils.convert_to_Tensor([_X_obs, _Z_obs, split_train, split_val, split_test])

Selecting 1 largest connected components
Number of trainging set: 124


/home/whb/anaconda3/envs/tf-gpu-cuda8/lib/python3.6/site-packages/scipy/sparse/compressed.py:708: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self[i, j] = values


In [4]:
# An instantiation of GSNN-M model
x_dim = feature_cuda.shape[1]
y_dim = label_cuda.shape[1]
h_dim, z_dim, r_dim = utils.get_h_dim(hidden_dim)

gcn = GCN(x_dim, h_dim, y_dim, True, False)
gsnn = GSNN(x_dim, y_dim, h_dim, r_dim, z_dim, gcn)

optimizer = torch.optim.Adam(gsnn.parameters(), lr=lr, weight_decay=weight_decay)
gsnntrainer = GSNNTrainer(gsnn, optimizer)

In [5]:
res = gsnntrainer.train(adj_cuda, feature_cuda, label_cuda, idx_train_cuda, idx_val_cuda, idx_test_cuda, epochs, earlystopping=earlystopping)

/home/whb/anaconda3/envs/tf-gpu-cuda8/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/whb/cx/GSNN-master/GSNN/GSNNTrain.py:102: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  kl_pq += (F.gumbel_softmax(y_encode[non_label].detach(), tau=1.0, hard=True) * (-F.log_softmax(y_pred_total[i][non_label]))).sum()/len(non_label)


Epoch: 0, Train_loss: 13.858444213867188.
--------------------------------------------------
Epoch: 10, Train_loss: 5.221202850341797.
--------------------------------------------------
Epoch: 20, Train_loss: 3.815887928009033.
--------------------------------------------------
Epoch: 30, Train_loss: 2.999713659286499.
--------------------------------------------------
Epoch: 40, Train_loss: 2.3898627758026123.
--------------------------------------------------
Epoch: 50, Train_loss: 2.207118034362793.
--------------------------------------------------
Epoch: 60, Train_loss: 2.0307657718658447.
--------------------------------------------------
Epoch: 70, Train_loss: 1.8835062980651855.
--------------------------------------------------
Epoch: 80, Train_loss: 1.9842537641525269.
--------------------------------------------------
Epoch: 90, Train_loss: 1.973174810409546.
--------------------------------------------------
Epoch: 100, Train_loss: 1.8750896453857422.
----------------------